<a href="https://colab.research.google.com/github/FranSC16/03MAIR---Algoritmos-de-Optimizacion/blob/main/AG2/Francisco_Santiburcio_Cort%C3%A9s_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#AG2 - Actividad Guiada 2
Nombre: Francisco Santiburcio Cortés

Link:   https://colab.research.google.com/drive/1bYEfPq5WuOXlpMV8ukn9StNXHEIsnbPc?usp=sharing


Github: https://github.com/FranSC16/03MAIR---Algoritmos-de-Optimizacion/blob/main/AG2/Francisco_Santiburcio_Cort%C3%A9s_AG2.ipynb





In [1]:
#Viaje por el rio - Programación dinámica
################################################################################

TARIFAS = [
[0,5,4,3,float("inf"),999,999],   #desde nodo 0
[999,0,999,2,3,999,11],  #desde nodo 1
[999,999, 0,1,999,4,10], #desde nodo 2
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]

In [10]:
#Calculo de la matriz de PRECIOS y RUTAS
#  PRECIOS - contiene la matriz del mejor precio para ir de un nodo a otro
#  RUTAS   - contiene los nodos intermedios para ir de un nodo a otro
################################################################  
def Precios(TARIFAS):
################################################################ 
  #Total de Nodos
  N = len(TARIFAS[0])
  
  #Inicialización de la tabla de precios
  PRECIOS = [ [9999]*N for i in [9999]*N]  #n x n 
  RUTA = [ [""]*N for i in [""]*N]
  
  #Se recorren todos los nodos con dos bucles(origen - destino)
  #  para ir construyendo la matriz de PRECIOS
  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
      
      for k in range(i, j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
            MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j] )
            RUTA[i][j] = k
        PRECIOS[i][j] = MIN
        
  return PRECIOS,RUTA

PRECIOS,RUTA = Precios(TARIFAS)

In [11]:
#Calculo de la ruta usando la matriz RUTA
def calcular_ruta(RUTA, desde, hasta):
  if desde == RUTA[desde][hasta]:
  #if desde == hasta:
    #print("Ir a :" + str(desde))
    return desde 
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(RUTA, 3,6) 


La ruta es:


3

In [13]:
#Asignacion de tareas - Ramificación y Poda
################################################################################
#    T A R E A 
#   A
#   G
#   E
#   N
#   T
#   E

COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]
        

In [45]:
#Calculo del valor de una solucion parcial
def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i] 
    # print("Agente: ",S[i]," Tarea: ",i," Costes: ",COSTES[S[i]][i]," Valor_acumulado: ",VALOR)
  return VALOR



valor((3,2,  ),COSTES)

67

In [15]:
#Coste inferior para soluciones parciales
#  (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range( len(S), len(COSTES)   ):
    VALOR += min( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range( len(S), len(COSTES)   ):
    VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

CI((0,1),COSTES)

68

In [16]:
#Genera tantos hijos como como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N ):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)    })
  return HIJOS

In [17]:
crear_hijos((0,) , 4)

[{'s': (0, 1)}, {'s': (0, 2)}, {'s': (0, 3)}]

In [2]:
def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(COSTES)
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)) )
  CotaSup = valor(MEJOR_SOLUCION,COSTES)
  #print("Cota Superior:", CotaSup)

  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((),COSTES)    } )

  iteracion = 0

  while( len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
    if len(NODO_FINAL ) >0: 
      #print("\n********Soluciones:",  [x for x in HIJOS if len(x['s']) == DIMENSION  ] )
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup   ]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor    ]
   
  print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
      


ramificacion_y_poda(COSTES)

NameError: ignored

# Apartado extra


In [42]:
import random
import time
import itertools
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

def fuerza_bruta(COSTES):
  mejor_valor = 0
  mejor_solucion = ()
  valores,soluciones = [],[]
  # itertools nos permite hacer todas las combinaciones posibles de agentes
  # Se asemeja a una tabla de verdad lógica
  for s in list(itertools.permutations(range(len(COSTES)))): 
    valor_aux = valor(s,COSTES)
    if valor_aux >= mejor_valor:
      mejor_valor = valor_aux
      mejor_solucion = s
      valores.append(mejor_valor)
      soluciones.append(mejor_solucion)
  print("valores ",valores," Soluciones ",soluciones)

  print("La solución óptima es: ",mejor_solucion," con valor",mejor_valor)


fuerza_bruta(COSTES)

Agente:  0  Tarea:  0  Costes:  11  Valor_acumulado:  11
Agente:  1  Tarea:  1  Costes:  15  Valor_acumulado:  26
Agente:  2  Tarea:  2  Costes:  19  Valor_acumulado:  45
Agente:  3  Tarea:  3  Costes:  28  Valor_acumulado:  73
Agente:  0  Tarea:  0  Costes:  11  Valor_acumulado:  11
Agente:  1  Tarea:  1  Costes:  15  Valor_acumulado:  26
Agente:  3  Tarea:  2  Costes:  20  Valor_acumulado:  46
Agente:  2  Tarea:  3  Costes:  23  Valor_acumulado:  69
Agente:  0  Tarea:  0  Costes:  11  Valor_acumulado:  11
Agente:  2  Tarea:  1  Costes:  17  Valor_acumulado:  28
Agente:  1  Tarea:  2  Costes:  13  Valor_acumulado:  41
Agente:  3  Tarea:  3  Costes:  28  Valor_acumulado:  69
Agente:  0  Tarea:  0  Costes:  11  Valor_acumulado:  11
Agente:  2  Tarea:  1  Costes:  17  Valor_acumulado:  28
Agente:  3  Tarea:  2  Costes:  20  Valor_acumulado:  48
Agente:  1  Tarea:  3  Costes:  22  Valor_acumulado:  70
Agente:  0  Tarea:  0  Costes:  11  Valor_acumulado:  11
Agente:  3  Tarea:  1  Costes: 

Como vemos arriba hay dos soluciones que son óptimas no solo una, pero escoge la primera que encuentra

**- Si generamos matrices de COSTES con valores aleatorios y de mayor dimensión y probamos ambos algoritmos**

In [49]:
import numpy as np
import time
import itertools

x,y = 5,5

COSTES = np.random.randint(1,50,(x,y))
start = time.time()
ramificacion_y_poda(COSTES)
mid_time = time.time()
fuerza_bruta(COSTES)
end = time.time()

print("Ramificación y poda: "+str(mid_time-start)+" s")
print("Fuerza Bruta: "+str(end-mid_time)+" s")

La solucion final es: [{'s': (4, 0, 1, 2, 3), 'ci': 81}]  en  74  iteraciones  para dimension:  5
valores  [97, 110, 114, 145, 158]  Soluciones  [(0, 1, 2, 3, 4), (0, 1, 3, 2, 4), (0, 1, 4, 2, 3), (0, 3, 1, 2, 4), (3, 0, 1, 2, 4)]
4
La solución óptima es:  (3, 0, 1, 2, 4)  con valor 158
Ramificación y poda: 0.007970809936523438 s
Fuerza Bruta: 0.0014331340789794922 s


El algoritmo de fuerza bruta funciona mejor con una matriz de 5x5, en comparación con el poda. El orden de complejidad del algoritmo de fuerza bruta es O(n), ya que solo tenemos un bucle, pero es posible que la pendiente de n sea grande por que dentro de dicho bucle se realizan bastante operaciones elementales

In [50]:
import numpy as np
import time
import itertools

x,y = 8,8

COSTES = np.random.randint(1,50,(x,y))
start = time.time()
ramificacion_y_poda(COSTES)
mid_time = time.time()
fuerza_bruta(COSTES)
end = time.time()

print("Ramificación y poda: "+str(mid_time-start)+" s")
print("Fuerza Bruta: "+str(end-mid_time)+" s")

La solucion final es: [{'s': (1, 3, 5, 0, 4, 6, 7, 2), 'ci': 61}]  en  829  iteraciones  para dimension:  8
valores  [150, 168, 204, 213, 222, 249, 260, 264, 265, 270, 273, 295, 306, 314, 320, 328, 330, 338]  Soluciones  [(0, 1, 2, 3, 4, 5, 6, 7), (0, 1, 2, 3, 4, 6, 5, 7), (0, 1, 2, 3, 4, 6, 7, 5), (0, 1, 2, 3, 5, 4, 6, 7), (0, 1, 2, 3, 5, 6, 7, 4), (0, 1, 2, 4, 3, 6, 7, 5), (0, 1, 2, 4, 5, 6, 7, 3), (0, 1, 3, 4, 5, 6, 7, 2), (0, 1, 4, 7, 5, 6, 3, 2), (0, 1, 5, 4, 2, 6, 7, 3), (0, 1, 5, 4, 3, 6, 7, 2), (0, 2, 1, 4, 3, 6, 7, 5), (0, 2, 1, 4, 5, 6, 7, 3), (0, 3, 1, 4, 5, 6, 7, 2), (1, 2, 0, 4, 5, 6, 7, 3), (1, 3, 0, 4, 5, 6, 7, 2), (4, 2, 1, 0, 5, 6, 7, 3), (4, 3, 1, 0, 5, 6, 7, 2)]
17
La solución óptima es:  (4, 3, 1, 0, 5, 6, 7, 2)  con valor 338
Ramificación y poda: 0.10994458198547363 s
Fuerza Bruta: 0.14078807830810547 s


A medida que aumentamos las dimensiones de la matriz de COSTES (8x8 en este caso) el algoritmo de ramificación y poda funciona mejor que el de fuerza bruta

In [ ]:
import numpy as np
import time
import itertools

x,y = 11,11

COSTES = np.random.randint(1,50,(x,y))
start = time.time()
ramificacion_y_poda(COSTES)
mid_time = time.time()
fuerza_bruta(COSTES)
end = time.time()

print("Ramificación y poda: "+str(mid_time-start)+" s")
print("Fuerza Bruta: "+str(end-mid_time)+" s")

Con unas dimensiones de 11x11 en la matriz de COSTES, mi ordenador no es capaz de terminar de ejecutar la celda. Es el tope de ramificación y poda

##Descenso del gradiente

In [ ]:
import math                      #Funciones matematicas
import matplotlib.pyplot as plt  #Generacion de gráficos (otra opcion seaborn)
import numpy as np               #Tratamiento matriz N-dimensionales y otras (fundamental!) 
#import scipy as sc

import random

In [ ]:
#Definimos la funcion
#Paraboloide
f  = lambda X:      X[0]**2 + X[1]**2    #Funcion
df = lambda X: [2*X[0] , 2*X[1]]         #Gradiente
 
df([1,2])

In [ ]:
from sympy import symbols
from sympy.plotting import plot
from sympy.plotting import plot3d
x,y = symbols('x y')
plot3d(x**2 + y**2,
       (x,-5,5),(y,-5,5),
       title='x**2 + y**2',
       size=(10,10))

In [ ]:
#Prepara los datos para dibujar mapa de niveles de Z
resolucion = 100 
rango=5.5
 
X=np.linspace(-rango,rango,resolucion) 
Y=np.linspace(-rango,rango,resolucion)
Z=np.zeros((resolucion,resolucion))
for ix,x in enumerate(X):
  for iy,y in enumerate(Y):
    Z[iy,ix] = f([x,y])

#Pinta el mapa de niveles de Z
plt.contourf(X,Y,Z,resolucion)
plt.colorbar() 

#Generamos un punto aleatorio inicial y pintamos de blanco
P=[random.uniform(-5,5  ),random.uniform(-5,5 ) ]
plt.plot(P[0],P[1],"o",c="white")

#Tasa de aprendizaje. Fija. Sería más efectivo reducirlo a medida que nos acercamos.
TA=.1 

#Iteraciones:50
for _ in range(50):
  grad = df(P)
  #print(P,grad)
  P[0],P[1] = P[0] - TA*grad[0] , P[1] - TA*grad[1]
  plt.plot(P[0],P[1],"o",c="red")

#Dibujamos el punto final y pintamos de verde
plt.plot(P[0],P[1],"o",c="green")
plt.show()
print("Solucion:" , P , f(P))
